# Анализ данных транзакций интернет-магазина


## Описание датасета

Датасет `default.retail` содержит информацию о транзакциях, совершенных в интернет-магазине. Каждая запись представляет собой отдельную транзакцию и включает следующие поля:

- **InvoiceNo**: Номер транзакции
- **StockCode**: Код товара
- **Description**: Описание товара
- **Quantity**: Количество единиц товара, добавленных в заказ
- **InvoiceDate**: Дата транзакции
- **UnitPrice**: Цена за единицу товара
- **CustomerID**: ID клиента
- **Country**: Страна, где проживает клиент

### 1. В каких странах имеется наибольшее число уникальных пользователей? 
   1. Постройте барчарт, где:
     - по оси x – страна
     - по оси y – число пользователей
   2. В качестве ответа укажите число уникальных пользователей из Великобритании.

```sql
SELECT  Country,
        COUNT(DISTINCT CustomerID) AS unique_customers
FROM default.retail
GROUP BY Country
ORDER BY unique_customers DESC 
LIMIT 10
```

![First Image](first.png)

### 2. Посмотрите на динамику изменения числа активных пользователей в месяц в Великобритании, Австралии и Нидерландах. Полученная вами результирующая таблица должна иметь вид: страна - число уникальных пользователей за определённый месяц. В качестве ответа укажите наименьшее число пользователей за февраль 2011.



```sql
SELECT  Country,
        toStartOfMonth(InvoiceDate) AS month,
        COUNT(DISTINCT CustomerID) AS customers
FROM default.retail
WHERE Country IN ('United Kingdom', 'Australia', 'Netherlands')
GROUP BY Country, month
ORDER BY month
```

![Second Image](second.png)

### 3. Давайте посмотрим на динамику изменения числа активных пользователей в месяц уже для всех стран, кроме Великобритании. 

```sql
SELECT  Country,
        toStartOfMonth(InvoiceDate) AS month,
        COUNT(DISTINCT CustomerID) AS customers
FROM default.retail
WHERE Country != 'United Kingdom'
GROUP BY Country, month
ORDER BY month
```

### В каких месяцах число активных пользователей во Франции было больше, чем в Германии?

```sql
SELECT  Country,
        toStartOfMonth(InvoiceDate) AS month,
        COUNT(DISTINCT CustomerID) AS customers
FROM default.retail
WHERE Country IN ('France', 'Germany')
GROUP BY Country, month
ORDER BY month
```

![Third Image](third.png)

Как видно на графике, число активных пользователей во Франции было больше, чем в Германии в мае, сентябре и декабре 2011 года.

### 4. Посчитайте среднюю сумму заказа (AOV – average order value) в каждой из стран.

```sql
WITH total_order_price AS
    (
SELECT  InvoiceNo,
        Country,
        SUM(UnitPrice * Quantity) AS total_price
FROM default.retail 
WHERE Quantity > 0
GROUP BY InvoiceNo, Country
    )

SELECT  Country,
        AVG(total_price) AS AOV 
FROM total_order_price
GROUP BY Country
ORDER BY AOV DESC
LIMIT 3
```

![Forth](forth.png)

### 5. Как изменялась средняя сумма заказа в разных странах по месяцам? Предположим, нас интересует динамика в следующих странах: United Kingdom, Germany, France, Spain, Netherlands, Belgium, Switzerland, Portugal, Australia, USA. 

```sql
WITH total_order_price AS
    (
SELECT  InvoiceNo,
        toStartOfMonth(InvoiceDate) AS month,
        Country,
        SUM(UnitPrice * Quantity) AS total_price
FROM default.retail 
WHERE Quantity > 0
GROUP BY InvoiceNo, month, Country
    )

SELECT  Country,
        month,    
        AVG(total_price) AS AOV 
FROM total_order_price
WHERE Country IN ('United Kingdom', 'Germany', 'France', 'Spain', 'Netherlands',
                  'Belgium', 'Switzerland', 'Portugal', 'Australia', 'USA')
GROUP BY Country, month
ORDER BY month ASC, AOV DESC
```

![Fifth Image](fifth.png)

#### Выводы по графику:
- Аномальный рост среднего чека произошел в августе 2011 среди пользователей из Австралии
- Меньше всего данных за этот период у США
- Самые большие значения среднего чека наблюдаются в Австралии и Нидерландах.


### 6. Сколько товаров пользователи обычно добавляют в корзину? Посчитайте среднее количество товаров, добавленных в корзину, с разбивкой по странам. 

В качестве ответа укажите среднее число товаров в заказе среди пользователей в Канаде (Canada).

```sql
WITH total_quantity AS
    (
SELECT  InvoiceNo,
        Country,
        SUM(Quantity) AS total_quantity
FROM default.retail 
WHERE Quantity > 0
GROUP BY InvoiceNo, Country
    )

SELECT  Country,
        AVG(total_quantity) AS avg_quantity 
FROM total_quantity
GROUP BY Country
```

![Sixth Image](sixth.png)

### 7. Определите, в каких странах совершается наибольшее количество заказов.

```sql
SELECT Country,
       COUNT(DISTINCT InvoiceNo) AS order_count
FROM default.retail
GROUP BY Country

```

![Seventh Image](seventh.png)

**Вывод**: Анализ количества заказов с разбивкой по странам выявил, что наибольшее количество заказов совершается в United Kingdom, Germany и France.


### 8. Произведите анализ выручки по дням недели.

```sql
SELECT DAYOFWEEK(InvoiceDate) AS day_of_week,
       SUM(UnitPrice * Quantity) AS total_revenue
FROM default.retail
GROUP BY day_of_week
ORDER BY day_of_week
```

![8 Image](8.png)

**Вывод**: Анализ выручки по дням недели показал, что чаще всего продажи происходят в четверг, при этом по субботам продаж не зафиксировано (0).